# Sanity Check
Just to check that we've got the same number of tokens when we tokenise our LSP words...

In [1]:
from transformers import AutoTokenizer
base_tok = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
big_tok = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-fr")

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd
#Converts data in src [TAB] tgt [NEWLINE] format to a format suitable for model training
def convertToDictFormat(data):
    source = []
    target = []
    for example in data:
        example = example.strip()
        sentences = example.split("\t")
        source.append(sentences[0])
        target.append(sentences[1])
    ready = Dataset.from_dict({"en":source, "fr":target})
    return ready
terms_in_train = load_dataset("ethansimrm/sampled_glossary_1.0_train", split = "train")
terms_in_train_ready = convertToDictFormat(terms_in_train['text'])
input = open("train_word_frequencies.txt", "r", encoding="utf8")
words = []
counts = []
for line in input.readlines():
    line_list = line.strip().split("\t")
    words.append(line_list[0])
    counts.append(int(line_list[1]))
input.close()
train_word_freq = pd.DataFrame(data = {"word":words, "count":counts}) #We could not use pandas' read_csv due to " being a token.
present = train_word_freq[train_word_freq["word"].isin(terms_in_train_ready['fr'])]
words_only = present["word"]

Found cached dataset text (C:/Users/ethan/.cache/huggingface/datasets/ethansimrm___text/ethansimrm--sampled_glossary_1.0_train-2f618074f2b08fee/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


In [3]:
len(present)

1406

In [4]:
words_only

5           chromosomique
87                 grippe
108             anticorps
112             détection
184            directives
               ...       
277240          Inférence
279574                Cor
283448      gatifloxacine
284551    Biostatistiques
285644           Parasite
Name: word, Length: 1406, dtype: object

In [5]:
words_only.to_csv("terms_for_lsp.txt", sep = "\n", header = False, index = False)

In [6]:
SPECIAL_BASE_TOKENS = [0, 1, 59513]
token_list = []
parent_word = []
counts = []
for row in present.itertuples():
    token_ids = base_tok(text_target=row.word)["input_ids"]
    for id in token_ids:
        if (id not in SPECIAL_BASE_TOKENS):
            token_list.append(id)

In [7]:
from collections import Counter
len(Counter(token_list))

1695